## MCP (Model Context Protocol)

### 1.1 MCP란 무엇인가?

MCP(Model Context Protocol)는  
LLM이 외부 시스템과 **표준화된 방식으로 상호작용**하도록 만든 프로토콜입니다.

기존 LLM의 한계는 명확합니다.

- 텍스트만 생성할 수 있습니다.
- 로컬 파일을 직접 다룰 수 없습니다.
- 외부 서비스(API, SaaS)를 직접 호출할 수 없습니다.

즉, 기존 LLM은 **읽고 말하는 도구**에 머물러 있었습니다.

MCP는 이 구조를 다음과 같이 확장합니다.

- 사용자가 자연어로 요청하면  
- LLM이 무엇을 해야 할지 **판단**하고  
- 실제 실행은 **외부 시스템(MCP Server)** 에 위임하며  
- 실행 결과를 다시 컨텍스트로 활용합니다.

핵심 요약은 다음 한 문장입니다.

> MCP는 LLM에게 **‘행동을 실행할 수 있는 표준 인터페이스’를 제공합니다.**

### 1.2 MCP의 3가지 핵심 구성 요소

MCP는 복잡해 보이지만, **'누가 시키고(Host)', '누가 연결하고(Client)', '누가 일하는지(Server)'**만 기억하면 간단합니다.

#### 1) MCP Host (명령하는 '두뇌')
- **역할:** **사용자와 직접 소통하는 애플리케이션**입니다. (예: Claude Desktop 앱, IDE 등)
- **기능:** 내장된 **LLM(두뇌)**을 사용하여 사용자의 요청을 분석하고, "어떤 도구가 필요한지" 판단하여 일을 시킵니다.
- **특징:** 직접 일을 처리하지 않고, 전체 흐름을 지휘하는 **컨트롤 타워** 역할을 합니다.

#### 2) MCP Client (연결하는 '신경망')
- **역할:** Host(두뇌)와 Server(손발) 사이를 **연결하는 중개자**입니다.
- **기능:** Host의 명령을 받아서 Server가 이해할 수 있는 **MCP 규약(프로토콜)**으로 번역해 전달하고, 실행 결과를 다시 Host에게 가져옵니다.
- **특징:** 공식 SDK(`mcp`)를 사용하여 안전한 통신 통로를 만듭니다.

#### 3) MCP Server (일하는 '손발')
- **역할:** 실제 **구체적인 작업**을 수행하는 기능의 집합입니다.
- **기능:** 파일 읽기, DB 저장, 검색, 외부 API 호출 등 실질적인 처리를 담당합니다.
- **특징:** LLM으로부터 실행 권한을 위임받은 **실제 백엔드**입니다.
    - *참고: 본 실습에서는 `fastmcp`를 사용해 이 부분을 직접 구현합니다.*


#### 핵심 원칙: "직접 하지 않는다"

초심자가 가장 주의해야 할 점은 **흐름의 순서**입니다.

> ❌ **LLM(Host)은 Server를 직접 건드리지 않습니다.**
>
> ⭕ **LLM(Host)** 이 판단하면 → **Client**가 신호를 보내고 → **Server**가 움직입니다.

**[비유로 요약]**
- **Host:** "주방장" (요리 주문을 받고 지시함)
- **Client:** "웨이터" (주방장의 지시를 각 요리 파트에 전달함)
- **Server:** "조리 도구/화구" (실제로 불을 켜고 재료를 볶음)

### 1.3 fastmcp란 무엇인가?

fastmcp는 **MCP Server를 가장 단순하게 구현할 수 있는 Python 라이브러리**입니다.

fastmcp의 특징은 다음과 같습니다.

- Python 함수 하나 = MCP Tool 하나
- MCP Tool 스키마 자동 생성
- STDIO 기반 실행 (로컬 실습에 최적)
- MCP Server 구현 복잡도 최소화

본 수업에서는 다음 원칙을 사용합니다.

> MCP Server 구현은 **fastmcp**  
> MCP Client 및 Host 구현은 **MCP 공식 SDK**

### 1.4 실습 시나리오  

#### 사용자 질문 → 모델 학습 결과(JSON) → Notion 자동 기록

#### 1.4.1 실습 목표

사용자는 다음과 같이 질문합니다.

> “train_result.json 파일을 요약해서  
> Notion에 기록해줘.”

이 한 문장으로 다음 작업이 자동 수행됩니다.

- 로컬 실험 결과 JSON 파일 읽기
- LLM이 실험 결과를 이해하고 요약
- 요약 결과를 Notion 페이지로 기록

이 실습의 핵심 목표는 다음과 같습니다.

> **자연어 → LLM 판단 → MCP Tool 실행 → 실제 외부 시스템 반영**  
> 이 한 사이클을 직접 완주하는 것입니다.

### 1.4.2 더미 실험 결과 파일 준비

로컬에 다음 파일을 생성합니다.

**파일명:** `train_result.json`

```json
{
  "epoch": 0.12,
  "global_step": 500,
  "loss": 0.3358,
  "learning_rate": 1.8812351543942996e-05,
  "total_flos": 265988832528000
}
```

### 1.4.3 실습 환경 준비 (라이브러리 설치)

아래 라이브러리를 추가로 설치합니다.

```bash
pip install fastmcp
pip install mcp
pip install notion-client
pip install langchain_mcp_adapters
```

In [1]:
from fastmcp import FastMCP
from dotenv import load_dotenv
from notion_client import Client
import json
import os

### 1.5 MCP Server 구현 (fastmcp)

#### 1.5.1 실험 결과 JSON 처리 MCP Server

다음 파일을 생성합니다.

**파일명:** `mcp_server.py`

```python
from fastmcp import FastMCP
from dotenv import load_dotenv
from notion_client import Client
import json
import os

load_dotenv()

mcp = FastMCP("ExperimentResultServer")

NOTION_TOKEN = os.getenv("NOTION_TOKEN")
NOTION_PAGE_ID = os.getenv("NOTION_PAGE_ID")

notion = Client(auth=NOTION_TOKEN)


@mcp.tool()
def read_experiment_result(file_path: str) -> dict:
    """
    모델 학습 결과 JSON 파일을 읽어 dict로 반환합니다.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)


@mcp.tool()
def upload_experiment_to_notion(title: str, summary: str) -> str:
    """
    요약된 실험 결과를 Notion 페이지로 업로드합니다.
    """
    notion.pages.create(
        parent={"page_id": NOTION_PAGE_ID},
        properties={
            "title": {
                "title": [
                    {"text": {"content": title}}
                ]
            }
        },
        children=[
            {
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [
                        {
                            "type": "text",
                            "text": {"content": summary}
                        }
                    ]
                }
            }
        ]
    )
    return "Notion 업로드 완료"


if __name__ == "__main__":
    print("🚀 Experiment MCP Server is running...")
    mcp.run(transport="stdio")
```


#### 1.5.2 MCP Server 실행

터미널에서 다음 명령을 실행합니다.

```bash
python mcp_server.py
```

정상 실행 시 서버는 **아무 작업도 하지 않은 채 대기 상태**에 있습니다.  
이는 오류가 아니라 **정상 동작 상태**입니다.

### 1.6 Notion 연동 준비

#### 1.6.1 Notion Integration Token 발급

1. https://www.notion.so/my-integrations 접속  

<img src="image/notion.png" width="650">

2. New Integration 생성- 이름 및 유형(프라이빗) 설정 
 
<img src="image/notion_2.png" width="650">

3. **Token** 복사하여 .env 파일에 NOTION_TOKEN 이름으로 저장

NOTION_TOKEN=ntn_XXXXXX

<img src="image/notion_3.png" width="650">

4. **사용권한** 탭에서  MCP 기능으로 활용할 페이지 설정  

MCP_TEST-2601 페이지 생성함 (선택)

웹 노션을 열어서 해당 페이지 URL 복사 (계정 아이콘 클릭 -> 노션 열기)


<img src="image/notion_4.png" width="650">

5. 해당 페이지에서 Page ID 복사하여 .env 파일에 NOTION_PAGE_ID 이름으로 저장

NOTION_PAGE_ID=XXXXXX

주의 ) https://www.notion.so/MCP_TEST_2601-2e58### 이후의 실제 페이지 id(2e58### 같은)만 포함하기

<img src="image/notion_5.png" width="650">


> ❗️주의  
> Page ID, API 토큰과 같은 **실행 세부 정보는 LLM에게 맡기지 않습니다.**  
> 이러한 정보는 MCP Server 내부 또는 환경변수로 관리합니다.

.env 파일 예시
```
OPENAI_API_KEY = sk-XXX
NOTION_TOKEN=ntn_XXX
NOTION_PAGE_ID=XXXX
```

### 1.7 MCP Host + Client 구현 (핵심)

이제 **사용자 질문 하나로 전체 흐름을 실행하는 MCP Host**를 구현합니다.

#### 1.7.1 MCP Host 코드

**파일명:** `mcp_host.py`

```python
import asyncio
from contextlib import AsyncExitStack
from dotenv import load_dotenv

from mcp import ClientSession
from mcp.client.stdio import stdio_client, StdioServerParameters

from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain.agents import create_agent

load_dotenv()


async def run():
    async with AsyncExitStack() as stack:
        # 1. MCP Server 연결
        params = StdioServerParameters(
            command="python",
            args=["mcp_server.py"]
        )

        read, write = await stack.enter_async_context(
            stdio_client(params)
        )

        session = await stack.enter_async_context(
            ClientSession(read, write)
        )
        await session.initialize()

        # 2. MCP Tool 로드
        tools = await load_mcp_tools(session)

        # 3. LLM 준비
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0
        )

        # 4. Agent 생성
        agent = create_agent(llm, tools)

        # 5. 사용자 질문
        user_question = """
        train_result.json 파일을 요약해서
        Notion에 기록해줘.
        제목은 '모델 실험 결과 - step 500'으로 해줘.
        """

        result = await agent.ainvoke({
            "messages": [
                ("user", user_question)
            ]
        })

        print(result)


if __name__ == "__main__":
    asyncio.run(run())
```

### 1.7.2 실행 순서 정리

1. 실험 결과 파일 생성 (`train_result.json`)
2. MCP Server 실행  
   ```bash
   python mcp_server.py
   ```
3. 다른 터미널에서 MCP Host 실행  
   ```bash
   python mcp_host.py
   ```

### 1.8 이 구조에서 중요한 점

- 사용자는 **자연어 질문만 입력**합니다.
- LLM은 무엇을 할지 **판단만** 합니다.
- 실제 실행은 MCP Server가 담당합니다.
- MCP Host가 전체 흐름을 제어합니다.

이 구조가 **정석적인 MCP 실행 구조**입니다.

### 1.9 요약

- MCP는 LLM을 “실행 주체”로 확장하는 프로토콜입니다.
- fastmcp는 MCP Server 구현을 단순화합니다.
- MCP Host는 판단, MCP Server는 실행을 담당합니다.
- 사용자 질문 하나로 실제 시스템 작업이 자동 수행됩니다.

## 2. 사용자 질문 → PDF 문서 요약 → Notion 자동 저장

이번 실습에서는  
앞선 JSON 실습을 확장하여 **비정형 문서(PDF)** 를 다룹니다.

핵심 구조는 바뀌지 않습니다.

- 사용자는 자연어로 요청하고  
- LLM은 무엇을 할지 판단하며  
- MCP Server는 실제 파일 처리와 저장을 수행합니다.

### 2.1 실습 목표

사용자는 다음과 같이 질문합니다.

> “농업기술.pdf 파일을 요약해서 Notion에 저장해줘.”

이 한 문장으로 다음 작업이 자동 수행됩니다.

- 로컬 PDF 파일 읽기
- PDF 텍스트 추출
- LLM 기반 문서 요약
- 요약 결과를 Notion 페이지로 기록

이번 실습의 목표는 다음과 같습니다.

> **PDF와 같은 비정형 문서도  
> MCP 구조로 문제없이 처리할 수 있음을 확인한다.**

### 2.2 실습 성공 기준

다음 조건을 만족하면 실습 성공입니다.

- 사용자는 자연어 질문만 입력한다
- PDF 파일을 직접 열지 않는다
- MCP Tool이 자동으로 호출된다
- Notion에 요약 페이지가 생성된다



### 2.3 실습용 PDF 파일 준비

요약 대상 PDF 파일을 준비합니다.

- 파일명 예시: `농업기술.pdf`
- 위치: `mcp_server.py`와 동일한 디렉토리

> PDF 파일 경로와 파싱 로직은  
> **LLM이 아닌 MCP Server에서 관리**합니다.



### 2.4 PDF 처리를 위한 라이브러리 설치

PDF 텍스트 추출을 위해 다음 라이브러리를 추가로 설치합니다.

```bash
pip install pypdf
``` 

### 2.5 MCP Server 확장 (PDF 처리 Tool 추가)

기존 MCP Server에 **PDF 전용 Tool**을 추가합니다.

#### 2.5.1 PDF 텍스트 추출 Tool

기존 `mcp_server.py`에 다음 코드를 추가합니다.

```python
from pypdf import PdfReader

@mcp.tool()
def read_pdf_file(file_path: str) -> str:
    """
    PDF 파일을 읽어 전체 텍스트를 반환합니다.
    """
    reader = PdfReader(file_path)
    pages = []

    for page in reader.pages:
        pages.append(page.extract_text())

    return "\n".join(pages)
```

이 Tool의 역할은 다음과 같습니다.

- PDF 구조 해석
- 페이지별 텍스트 추출
- 요약이나 판단은 수행하지 않음

> 파일 파싱은 MCP Server,  
> 문서 이해와 요약은 LLM의 역할입니다.

### 2.6 MCP Server 역할 정리

이제 MCP Server는 다음 기능을 제공합니다.

- JSON 파일 읽기
- PDF 파일 텍스트 추출
- Notion 페이지 생성 및 저장

파일 형식이 달라져도  
**MCP Server의 책임은 변하지 않습니다.**

### 2.7 MCP Host 실행 (PDF 요약 요청)

MCP Host(`mcp_host.py`) 코드는 **변경하지 않습니다.**

사용자 질문만 다음과 같이 변경합니다.

```text
data/농업기술.pdf 파일을 요약해서 Notion에 저장해줘.

제목은 "농업기술 정책 문서 요약"으로 해줘.
```

LLM은 내부적으로 다음을 수행합니다.

1. PDF 파일을 읽어야 한다고 판단
2. `read_pdf_file` MCP Tool 호출
3. 추출된 텍스트를 요약
4. Notion 저장 Tool 호출

### 2.8 설계 관점에서의 핵심 포인트

- PDF 파싱은 LLM에게 맡기지 않는다
- MCP Server는 실행만 담당한다
- LLM은 판단과 요약만 수행한다
- JSON → PDF로 바뀌어도 구조는 동일하다

이 점이 **MCP 구조의 가장 큰 장점**입니다.

### 2.9 요약

- MCP는 비정형 문서(PDF) 처리에도 적합하다
- PDF 처리는 MCP Server가 담당한다
- LLM은 문서 요약과 판단에 집중한다
- 사용자 질문 하나로 PDF 요약 자동화가 가능하다
